In [222]:
# prompt = r"""
# THe program doesn't output anything in windows 10.
# (base) C:\Users\off99\Documents\Code\>llm list files in current dir; windows
# / Querying GPT-3200
# ───────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────
#        │ File: temp.sh
# ───────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────
#    1   │
#    2   │ dir
#    3   │ ```
# ───────┴────────────────────────────────────────────────────────────────────────────────────────────────────────────────
# >> Do you want to run this program? [Y/n] y

# Running...


# (base) C:\Users\off99\Documents\Code\>
# Notice that there is no output. Is this supposed to work on Windows also?
# Also it might be great if the script detects which OS or shell I'm using and try to use the appropriate command e.g. dir instead of ls because I don't want to be adding windows after every prompt.
# """
# repo_url = "https://github.com/jayhack/llm.sh"


# prompt = "convert project to typescript"
# repo_url = "https://github.com/andrewagain/calculator"


prompt = """ 
I just created a new project! It is called Tiny Gen. Is it possible to add this project to portfolio site? 
Tiny Gen is basically an application that can add code to a repository given a user prompt. Basically, you give a repoUrl and the prompt,
Tiny Gen is able to locate the file to modify and modifies it! It then returns a unified diff showing the correct changes. 

I'll upload the links, images and other information later.
"""
repo_url = "https://github.com/tawsifkamal/portfolio-site"


In [240]:
from db import fetch_all_calls
from db import insert_to_supabase

fetch_all_calls("tiny_gen_two_calls")

2024-03-02 18:14:01,177:INFO - HTTP Request: GET https://pzgzqbmlwjfvjtsziwdb.supabase.co/rest/v1/tiny_gen_two_calls?select=%2A "HTTP/1.1 200 OK"


'{"data":[],"count":null}'

### Tiny Gen 1.0: Works for Repos that fit in the context
1. **Code Conversion Chain** -> Given prompt and all of files in the github repo, generate the code that would address the prompt given

2. **Generate Diff Chain** -> Generate the diff from the modified code versus the original file

2. **Reflection Chain** -> Given the generated diff, check that the diff is correct and return a new diff if incorrect


In [225]:
from tiny_gen_one import TinyGenOne

tiny_gen = TinyGenOne()
result = tiny_gen.call(repoUrl=repo_url, prompt=prompt)
print(result)

CURRENT RATE LIMIT FOR GITHUB API CALLS >>>>>>>>>>> {'resources': {'core': {'limit': 5000, 'used': 301, 'remaining': 4699, 'reset': 1709419058}, 'search': {'limit': 30, 'used': 0, 'remaining': 30, 'reset': 1709418565}, 'graphql': {'limit': 5000, 'used': 0, 'remaining': 5000, 'reset': 1709422105}, 'integration_manifest': {'limit': 5000, 'used': 0, 'remaining': 5000, 'reset': 1709422105}, 'source_import': {'limit': 100, 'used': 0, 'remaining': 100, 'reset': 1709418565}, 'code_scanning_upload': {'limit': 1000, 'used': 0, 'remaining': 1000, 'reset': 1709422105}, 'actions_runner_registration': {'limit': 10000, 'used': 0, 'remaining': 10000, 'reset': 1709422105}, 'scim': {'limit': 15000, 'used': 0, 'remaining': 15000, 'reset': 1709422105}, 'dependency_snapshots': {'limit': 100, 'used': 0, 'remaining': 100, 'reset': 1709418565}, 'audit_log': {'limit': 1750, 'used': 0, 'remaining': 1750, 'reset': 1709422105}, 'code_search': {'limit': 10, 'used': 0, 'remaining': 10, 'reset': 1709418565}}, 'rate

### Tiny Gen 2.0: Works for Repos longer than LLM Context
1. **Summary Chain** --> summarize all files

2. **Identify Relevant Files Chain** --> given a summary of each file, identify relevant files to the user prompt 

3. **Code Conversion Chain** -> Given the prompt and summaries of files, generate the code to address the prompt

4. **Generate Diff Chain** -> Generate the diff from the modified code versus the original file

5. **Reflection Chain** -> Given the generated diff, check that the diff is correct and return a new diff if incorrect

In [221]:
from tiny_gen_two import TinyGenTwo

tiny_gen = TinyGenTwo()
result = tiny_gen.call(repoUrl=repo_url, prompt=prompt)
print(result)

CURRENT RATE LIMIT FOR GITHUB API CALLS >>>>>>>>>>> {'resources': {'core': {'limit': 5000, 'used': 153, 'remaining': 4847, 'reset': 1709419058}, 'search': {'limit': 30, 'used': 0, 'remaining': 30, 'reset': 1709417627}, 'graphql': {'limit': 5000, 'used': 0, 'remaining': 5000, 'reset': 1709421167}, 'integration_manifest': {'limit': 5000, 'used': 0, 'remaining': 5000, 'reset': 1709421167}, 'source_import': {'limit': 100, 'used': 0, 'remaining': 100, 'reset': 1709417627}, 'code_scanning_upload': {'limit': 1000, 'used': 0, 'remaining': 1000, 'reset': 1709421167}, 'actions_runner_registration': {'limit': 10000, 'used': 0, 'remaining': 10000, 'reset': 1709421167}, 'scim': {'limit': 15000, 'used': 0, 'remaining': 15000, 'reset': 1709421167}, 'dependency_snapshots': {'limit': 100, 'used': 0, 'remaining': 100, 'reset': 1709417627}, 'audit_log': {'limit': 1750, 'used': 0, 'remaining': 1750, 'reset': 1709421167}, 'code_search': {'limit': 10, 'used': 0, 'remaining': 10, 'reset': 1709417627}}, 'rate

# Idea for Tiny Gen 3.0
### 3. Tiny Gen 3.0 Works for Repos longer than LLM Context + Modifying More Than One File
 1. **Summary Chain** --> summarize all files 

 2. **Identify Relevant Files Chain** --> given a summary of each file, identify relevant files to the user prompt

 3. **Breakdown Task Chain** --> given the readme, relevant files, and user query, breakdown the user prompt into smaller tasks

 4. **Code Conversion Chain** --> For each task, generate the corresponding code changes to fulfill the task

 4. **Generate Diffs Chain** --> For each code change, generate the associated diff from the original file and modified file
 
 5. **Reflection Chain** --> For each generated diff, check if it's correct! If not, regenerate and output the new one. If correct, output the original one
 